In [1]:
### ZERO-SHOT CLASSIFICATION, TOPIC LABELLING ###

# TODOs: Discovering Interpretable Topics by Leveraging Common Sense Knowledge (Facebook)
# TODOs: https://towardsdatascience.com/zero-shot-text-classification-with-hugging-face-7f533ba83cd6
# DONE: Extract args, ids for labelling
# DONE: Get Labels
# TODOs: NLI vs Semantic
# TODOs: Zero-shot Classification ConceptNet (semantic extension)
# TODOs: Expand: Concept Net synonyms

In [2]:
### ASSERT WORKING DIRECTORY FOR IMPORTS ###
import os
print(os.getcwd())

/Users/joshua.sheppard/PycharmProjects/countaBot/src/detection


In [3]:
import pandas as pd
import json
import random
from tqdm.notebook import tqdm

In [4]:
### LOAD ###
#args = [json.loads(ln) for ln in open("../data/cmv_processed.jsonl")]
ids = [json.loads(ln)["id"] for ln in open("../data/cmv_processed.jsonl")]
args = [json.loads(ln)["argument"]["argument"] for ln in open("../data/cmv_processed.jsonl")]

claims = [json.loads(ln)["claim"] for ln in open("../data/cmv_processed.jsonl")]
#args = [json.loads(ln) for ln in open("../data/cmv_processed.jsonl")]
#len(args)

In [5]:
# for j, k in zip(args, ids):
#     if j == "":
#         print("blanks", j, k)

In [6]:
### SAMPLE ###
_ = random.randint(0, len(args))
args[_]

'Honestly I dont know where all the hate for Hillary comes from sure I dont love her as a candidate and I disagree with some of her stances but most of the stuff floating around about her seems pretty contrived. Starting small a lot of people get angry about her statements on the keystone pipeline. She changed her mind! She wanted to support Obama but when all the negative attention starting popping up she flip flopped and opposed it. Would you rather she had stuck with it?'

In [7]:
### UNIQUE ARGUMENTS ###
corpus = set()
idx = set()

for j, k in zip(args, ids):
    corpus.add((j, k))

corpus = list(corpus)
type(corpus), len(corpus)

(list, 5990)

In [36]:
### KEYPHRASE LIST ###
kps = [json.loads(ln)["argument"]["arg_keyphrases"] for ln in open("../data/cmv_processed.jsonl")]

### UNIQUE KP-ARGUMENTS ###
corpus_kp = set()
idx = set()

for j, k in zip(kps, ids):
    kps = ", ".join(i for i in j)
    corpus_kp.add((kps, k))

corpus_kp = list(corpus_kp)
type(corpus_kp), len(corpus_kp)

(list, 5990)

In [8]:
### UNIQUE CLAIMS ###
# claim_corpus = set()
# idx = set()
#
# for j, k in zip(claims, ids):
#     claim_corpus.add((j, k))
#
# claim_corpus = list(claim_corpus)
# type(claim_corpus), len(claim_corpus)

In [9]:
# claim_corpus

In [10]:
### EXTRACT UNIQUE SET ###
# import pandas as pd
#
# def unique_entries(args, key="id"):
#     data_ = pd.DataFrame(args)
#     unique = data_.drop_duplicates(subset=key)
#
#     unique_ = []
#     for _, i in unique.iterrows():
#         unique_.append({
#             "id": i["id"],
#             "argument": i["argument"],
#             "tgt_counter": i["counter"]
#         })
#
#     return unique_
#
# unique_args = unique_entries(args)
# len(unique_args)

In [21]:
### COMMON-SENSE POLARISED TOPIC LABELS ###
import re
def clean(clean):
    clean = re.sub(r"\n", "", clean)
    clean = re.sub(r'(?<=[a-z])\'(?=[a-z])', '', clean)
    clean = re.sub('([^a-zA-Z\s.!?])', "", clean)
    clean = re.sub('\s+', ' ', clean)

    clean = re.sub(r"www\S+", "", clean)
    return clean.strip().lower()

#TODOs: Prune and Add CauseNet Topics
controversial_topics = [clean(ln) for ln in open("../data/concepts/wiki_controversial_topics.txt")]
debate_topics = [clean(ln) for ln in open("../data/concepts/IBM_debate_topics_I.txt")]
debate_topics_ = [clean(ln) for ln in open("../data/concepts/IBM_debate_topics_II.txt")]
arg_kb_20 = [clean(ln) for ln in open("../data/concepts/argkb_20_topics.txt")]
wiki_race = [clean(ln) for ln in open("../data/concepts/wiki_racism_related.txt")]
wiki_ideologies = [clean(ln) for ln in open("../data/concepts/wiki_political_ideologies.txt")]

concepts = [clean(ln) for ln in open("../data/concepts/cause_concepts.txt")]

topics = []
topics.extend(controversial_topics)
topics.extend(debate_topics)
topics.extend(debate_topics_)
topics.extend(wiki_ideologies)
topics.extend(arg_kb_20)
topics.extend(wiki_race)

len(topics)

6521

In [22]:
topics

['invasion of iraq',
 'abortion',
 'affirmative action',
 'african national congress',
 'alberta separatism',
 'american civil liberties union',
 'american hunters and shooters association',
 'history of the jews in the united states',
 'american jews',
 'anarchism',
 'anarchocapitalism',
 'antiamericanism',
 'anticlericalism',
 'antiirish racism',
 'antisemitism',
 'asian american',
 'atheism',
 'austrian school',
 'bashar assad',
 'syrian civil war',
 'bidoon social class',
 'black lives matter',
 'black supremacy',
 'boricua popular army',
 'brexit',
 'british national party',
 'capitalism',
 'capital punishment',
 'catalonia',
 'cbc news',
 'cherokee',
 'china',
 'east turkestan independence movement',
 'tibetan independence movement',
 'human rights in china',
 'chinese intelligence activity abroad',
 'christian right',
 'plame affair',
 'plame affair timeline',
 'communism',
 'communist party of china',
 'communist government',
 'communist state',
 'cnn',
 'conservatism in the un

In [37]:
# Semantic-Search, Cosine Similarity
from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer('all-MiniLM-L6-v2')

def semantic_search(corpus, query, threshold=0.15):
    # Construct Corpus set
    corpus_, id_ = zip(*corpus)
    corpus_ = list(corpus_)
    id_ = list(id_)

    # Embed the Corpus
    corpus_embeddings = embedder.encode(corpus_, convert_to_tensor=True)

    # Construct Query-Label set
    queries = set(query)

    mapped_dict = {}
    for i in range(0, len(id_)):
        mapped_dict[id_[i]] = {"argument": corpus_[i], "label": []}

    # Return top k=1 argument for each Label via Cosine Similarity
    top_k = min(1, len(corpus_))

    with tqdm(total=len(queries)) as pbar:
        for query in queries:
            query_embedding = embedder.encode(query, convert_to_tensor=True)

            cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
            top_results = torch.topk(cos_scores, k=top_k)

            for score, idx in zip(top_results[0], top_results[1]):
                # 'Empirical' threshold
                if score >= threshold:
                    # Append Label
                    #mapped_dict[id_]["argument"] = corpus_[idx]
                    # Note: Can use the same idx index
                    mapped_dict[id_[idx]]["label"] = query.lower()

                #else: mapped_dict[id_]["label"] = "None"

            pbar.update()

    return mapped_dict

In [38]:
### EXTRACT LABELLED INSTANCES ###
def extract_labelled(dict_):
    res = []
    for i in dict_.items():
        j, k = i

        if k["label"] != []:
            res.append({"id": i[0], "argument": k["argument"], "label": k["label"]})
    return res

In [39]:
# TODOs: Reverse the Query
#sample = corpus[0:100]

### LABELLING: CONTROVERSIAL TOPICS ###
kp_topics = semantic_search(corpus=corpus_kp, query=topics, threshold=0)

  0%|          | 0/5674 [00:00<?, ?it/s]

In [40]:
print(len(kp_topics), len(corpus))
print(len(kp_topics), len(extract_labelled(kp_topics)))
kp_topics

5990 5990
5990 2493


{'t3_2gnq5k': {'argument': 'easy conversion, thousand paces long, benefit, conversion',
  'label': []},
 't3_1x015l': {'argument': 'Christmas we couldnt, Santa Claus, younger siblings, siblings the youngest',
  'label': 'sibling'},
 't3_5wqh22': {'argument': 'hotbutton topic, intelligent method, Drone warfare, recriminations should fall',
  'label': 'drone strike'},
 't3_2icu72': {'argument': 'mom, addressed her mother, dad, parents',
  'label': 'mothers'},
 't3_3izsdq': {'argument': 'suboptimal decisions, huge loans, huge sums, higher education',
  'label': []},
 't3_4i6sxi': {'argument': 'state, grew, largest, city',
  'label': 'state economy'},
 't3_3m2sw7': {'argument': 'hidden agenda, wears his skeletons, guy, honor',
  'label': []},
 't3_2vwn9m': {'argument': 'make a facebook, Star Wars shirt, facebook post, free speech.Both Jesus',
  'label': []},
 't3_2qncj3': {'argument': 'Recently Ive disagreed, men in uniform, Ive always wanted, Ive',
  'label': []},
 't3_42ra9b': {'argument

In [29]:
# TODOs: Reverse the Query
#sample = corpus[0:100]

### LABELLING: CONTROVERSIAL TOPICS ###
arg_topics = semantic_search(corpus=corpus, query=topics, threshold=0)

  0%|          | 0/5674 [00:00<?, ?it/s]

In [26]:
print(len(arg_topics), len(corpus))
print(len(arg_topics), len(extract_labelled(arg_topics)))
arg_topics

5990 5990
5990 2285


{'t3_3normn': {'argument': 'In the Harry Potter series Hogwarts students are sorted into one of four houses named for the founding members of the school on the basis of particular qualities favoured by each of those founding members.Gryffindor is the house for the good heroic and brave.Ravenclaw is the house for the studious and academic.Hufflepuff is the house for the helpful and reliable.Slytherin is the house for the conniving and ambitious.Now the students have some level of choice over the house they enter Harry proves as much. However there is clearly some kind of systematic allocation as all of the evil ambitious wizards and witches are sorted into Slytherin and all of the great and heroic ones who fight them are sorted into Gryffindor over the entire history of the school. That may be because those students aspire to those features already or because they dont have as much choice as we think.Either way my view is that this method of sorting students is terrible and most likely 

In [59]:
# TODOs: Reverse the Query
#sample = corpus[0:100]

### LABELLING: CONTROVERSIAL TOPICS ###
claim_topics = semantic_search(corpus=claim_corpus, query=topics, threshold=0)

  0%|          | 0/1537 [00:00<?, ?it/s]

In [60]:
print(len(claim_topics), len(claim_corpus))
print(len(arg_topics), len(extract_labelled(claim_topics)))
arg_topics

5990 5990
5990 894


{'t3_1er14x': {'argument': 'In almost any thread or conversation about ProChoice vs ProLife a ProChoice person will say that No ones ProAbortion or I personally wouldnt get an abortion but I think it should be a choice. I honestly have no idea how this position makes any sense. Even though Im ProLife I can follow the logic of a person who admits to being ProAbortion and even in a sense respect that that is their opinion. I just cannot grasp why other people always call it ProChoice when in the end it boils down to ProAbortion.',
  'label': []},
 't3_55a3bd': {'argument': 'Let me start by saying I think we need to elect Hillary Clinton. I am no fan of Clinton but I know her to be the lesser of two inevitable evils. Jill Stein and Gary Johnson are not legitimate options and they present a real spoiler threat to Hillarys chances. We live under a pluralityrule first past the post electoral system that invariably marginalizes third parties. I am not a political scientist or a statistician b

In [13]:
### LABELLING: CONCEPTS ###
arg_concepts = semantic_search(corpus=corpus, query=concepts)

  0%|          | 0/51359 [00:00<?, ?it/s]

In [14]:
len(arg_concepts), len(extract_labelled(arg_concepts))

(5990, 4093)

In [132]:
### STORE TOPIC LABELS CONCEPTS AND LABELS ###
fout = open("../data/argument_topic_concept.jsonl", "w")

for j, k in arg_topics.items():
    fout.write(json.dumps({
        "id": j,
        "argument": k["argument"],
        "topic_label": k["label"],
        #"concept_label": k["label"]
    }))
    fout.write("\n")

In [16]:
### STORE CONCEPT LABELS CONCEPTS AND LABELS ###
fout = open("../data/argument_concepts.jsonl", "w")

for j, k in arg_concepts.items():
    fout.write(json.dumps({
        "id": j,
        "argument": k["argument"],
        "concept_label": k["label"],
    }))
    fout.write("\n")

In [127]:
for j, k in arg_topics.items():
    if k["argument"] == "":
        print("blanks", j)

blanks t3_1egv4k
blanks t3_3cm6jy
blanks t3_5wjdve


In [103]:
arg_concepts

{'t3_6dj73v': {'argument': 'Pizza is wonderful. Top food stuff of mine easily but I do not understand the trend for thicker and thicker bases. I think of the base of a pizza as a vector for getting the delicious flavors of the sauce and the cheese and the toppings into my mouth. I dont want that wonderful tomato sauce goodness overpowered by soggy tasteless bread. The thinner base also means the pizza is crispier and you dont need to support the end of the slice with your other hand.',
  'label': []},
 't3_6i4hip': {'argument': 'Just a heads up Im gonna curse for a minute.I have been a hardcore gamer for the past ten years but my discourse with internet communities surrounding the games I play really began when I started playing League of Legends around years ago. Over those four years I have become more and more skeptical and ultimately completely disdainful of not only the people who play the online games I do but in general the gaming publics absolutely childlike reactions towards g

In [ ]:
### DOMAIN RESTRICTED ###



In [392]:
# ### QUERY EXPANSION ###
#
# # TODOs: Query Expansions [TypeOf, SimilarTerms, CanBe]
# # https://github.com/fitosegrera/python-conceptnet/blob/master/ConceptNet.py
# import json
# import urllib
#
# URL = "http://api.conceptnet.io/"
#
# # TODOs: Review. Similarity.
# class ConceptNet:
#
#     def __init__(self, api, l):
#         self.api = api
#         self.l = l
#
#     def search(self, lang, term):
#         url_to_search = self.api + "c/" + lang + "/" + term
#         data = urllib.request.urlopen(url_to_search)
#         json_data = json.load(data)
#         for i in json_data["edges"]:
#             print("----------------")
#             print(i["end"])
#             print("relation:", i["rel"])
#             print(i["surfaceEnd"])
#             print(i["surfaceStart"])
#             print("weight:", i["weight"])
#
#         return json_data
#
#     def get_relation(self, rel, concept):
#         url_to_search = self.api + f"search?node=/c/en/{concept}&rel=/r/{rel}"
#         data = urllib.request.urlopen(url_to_search)
#         obj_ = json.load(data)
#
#         labels = set()
#         for _ in obj_["edges"]:
#             labels.add((_["end"]["label"], _["weight"]))
#
#         return labels
#
#     def get_similar(self, concept):
#         res = []
#         rels = ["Synonym", "SimilarTo"]
#         for _ in rels:
#             res.extend(self.get_relation(_, concept))
#
#         return sorted(res, key=lambda x: x[1], reverse=True)[:self.l]

In [393]:
# concept_net = ConceptNet(api=URL, l=3)

In [90]:
### CONCEPT NET ###
# from src.utils_.concept_net_expansion import ConceptNet
# URL = "http://api.conceptnet.io/"
#
# concept_net = ConceptNet(URL, l=3)
# expansion_term = "carriage"
#
# test_concept = concept_net.get_similar(expansion_term)
# test_concept

[('carriage', 2.0), ('coach', 1.0), ('carriage', 1.0)]

In [404]:
### WORDNET ###
# from src.utils_.word_net_expansion import expand_query
#
# test_wn = expand_query(expansion_term)
# test_wn

'fabulous fab mythic'

In [134]:
### EXTRACT LABELLED INSTANCES ###
# def extract_labelled(dict_):
#     res = []
#     for i in dict_.items():
#         j, k = i
#
#         if k["label"] != "None":
#             res.append({"id": i[0], "argument": k["argument"], "label": k["label"]})
#     return res

# args_topic_labelled = extract_labelled(arg_topics)
# print(len(args_topic_labelled))
#
# args_topic_labelled

In [135]:
# args_topic_labelled = extract_labelled(arg_topics)
# print(len(args_topic_labelled))
#
# args_topic_labelled

4782


[{'id': 't3_6d8okw',
  'argument': 'Casebycase schools that are largely successful are correlatively successful with their local schools compared to national peers. The mindset of the community matters. Many attribute the ailing innercity schools to cultural issues and biases having worked with innercity populations for five years and having worked with hundreds of students perfectly capable of rational thought and argument that nevertheless perform poorly I agree. In general American culture devalues intelligence some areas more than others. Literacy movements are wonderful but until people stop seeing learning as lame or avoiding intellectual discourse this wont change.',
  'label': 'Year-round school in the United States'},
 {'id': 't3_25fjdx',
  'argument': 'Ah like it says in Dust In The Wind. Every line in that song is correct except for nothing lasts forever but the earth and sky. Probably wont last forever either.I believe that it doesnt matter what you do in life. Well not in 